In [ ]:
import gromacs as gmx
import numpy as np
import mdtraj as md
import nglview as nv

In [ ]:
dir='/home/meta/ljocha/ASMSA/My_AE/trpcage/ae'
ftrain=dir + '/trpcage_ds.npy'
pdb = dir + '/trpcage_npt400.pdb'
dec = '/home/meta/ljocha/ASMSA/My_AE/trpcage/dec_out_LS_matrix_pos_32x32.npy'

train = np.load(ftrain)

In [ ]:
train_view = np.reshape(train,(-1,272,3))

In [ ]:
train.shape

In [ ]:
tr = md.load(pdb)

In [ ]:
tr.xyz = train_view

In [ ]:
v=nv.show_mdtraj(tr)
v

In [ ]:
dec_out_all = np.load(dec)
dec_out_all.shape

In [ ]:
def Unstandardize(x):
    x = np.array(x)
    return ((x*7)-1)

In [ ]:
dec_out_all = Unstandardize(dec_out_all)

In [ ]:
dec_out = dec_out_all[0,0]

In [ ]:
diff = train - dec_out
msd = np.sum(diff * diff,axis=1)

In [ ]:
np.argmin(msd)

In [ ]:
tr2 = md.load(pdb)
tr2.xyz = np.reshape(dec_out,(-1,272,3))
tr2.xyz = train_view[9873]
v2 = nv.show_mdtraj(tr2)
#v2.clear()
#v2.add_representation("licorice")
v2

In [ ]:
msd[1234]

In [ ]:
np.reshape(dec_out,(-1,272,3)).shape

In [ ]:
v.frame=1234

In [ ]:
v.frame=3702

In [ ]:
diff.shape

In [ ]:
msd.shape

In [ ]:
np.random.normal(size=dec_out.shape)